In [ ]:
import numpy as np
import pandas as pd
import os
import random
import cv2
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [ ]:
data0 = pd.read_csv('../input/tmnist-glyphs-1812-characters/Glyph_TMNIST.csv')
data1=data0[['font_name','label']]
data2=data1.groupby('font_name',as_index=False).sum()
data2

In [ ]:
for i in range(50):     # upto 2990
    print(data2.iloc[i,0])
    print(data2.iloc[i,1])
    print()

In [ ]:
del data1
del data2

In [ ]:
from tqdm import tqdm
drop=[]
for i in tqdm(range(len(data0))):
    item=data0['label'][i]
    if item not in 'ČĆĐŠŽčćđšž₹ĂÂÊÔƠƯăâêôơưАБВГҐДЂЕЁЄЖЗЅИІЇЙЈКЛЉМНЊОПРСТЋУЎФХЦЧЏШЩЪЫЬЭЮЯабвгґдђеёєжзѕиіїйјклљмнњопрстћуўфхцчџшщъыьэюя':
        drop+=[i]

In [ ]:
data5=data0.drop(index=data0.index[drop],axis=0).reset_index(drop=True)
data5

In [ ]:
del data0

In [ ]:
Name=[]
char='ČĆĐŠŽčćđšž₹ĂÂÊÔƠƯăâêôơưАБВГҐДЂЕЁЄЖЗЅИІЇЙЈКЛЉМНЊОПРСТЋУЎФХЦЧЏШЩЪЫЬЭЮЯабвгґдђеёєжзѕиіїйјклљмнњопрстћуўфхцчџшщъыьэюя'
for i in range(len(char)):
    Name+=[char[i]]
print(Name)
print(len(Name))

In [ ]:
M=list(range(len(Name)))
normal_mapping=dict(zip(Name,M)) 
reverse_mapping=dict(zip(M,Name)) 

In [ ]:
n=len(data5)
trainY=data5['label'][0:(n//4)*3].map(normal_mapping).reset_index(drop=True)
testY=data5['label'][(n//4)*3:].map(normal_mapping).reset_index(drop=True)

In [ ]:
data6=data5.drop(['glyph_name','font_name','label'],axis=1)
trainX=data6.loc[0:(n//4)*3-1].reset_index(drop=True)
testX=data6.loc[(n//4)*3:].reset_index(drop=True)

In [ ]:
del data5

In [ ]:
binencoder = LabelBinarizer()
y0 = trainY
y = binencoder.fit_transform(y0)
y

In [ ]:
X_images = trainX.values.reshape(-1,28,28)
test_images = testX.values.reshape(-1,28,28)
print(X_images.shape)
print(test_images.shape)

In [ ]:
fig, axs = plt.subplots(3,3,figsize=(12,12))
for i in range(9):
    r=i//3
    c=i%3
    ax=axs[r][c].axis("off")
    actual=reverse_mapping[trainY[i]]     
    ax=axs[r][c].set_title(actual)
    ax=axs[r][c].imshow(X_images[i])
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_images, y, test_size=0.2, random_state=90)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
X_train = (X_train/255).reshape(-1,28,28,1).astype('float32')
X_test = (X_test/255).reshape(-1,28,28,1).astype('float32')

In [ ]:
test_images = (test_images/255).reshape(-1,28,28,1).astype('float32')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.callbacks import EarlyStopping
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

In [ ]:
model = Sequential()
model.add(Conv2D(32,(4,4),input_shape = (28,28,1),activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3),activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(len(Name)*4,activation='relu'))
model.add(Dense(len(Name)*2, activation='relu'))
model.add(Dense(len(Name), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
result = model.fit(X_train, y_train, validation_split=0.2, epochs=20, batch_size=92, verbose=2)
result

In [ ]:
his = pd.DataFrame(result.history)
his.loc[:, ['accuracy', 'val_accuracy']].plot()
his.loc[:, ['loss', 'val_loss']].plot()

In [ ]:
y_pred = model.predict(test_images)
print(y_pred[0])

In [ ]:
y_pred2=[]
for item in y_pred:   
    y_pred2+=[np.argmax(item)   ]
PRED=np.array(y_pred2)
ANS=np.array(testY)
print(PRED[0:10])
print(ANS[0:10])

In [ ]:
accuracy_score(ANS,PRED)

In [ ]:
fig, axs = plt.subplots(3,3,figsize=(12,12))
for i in range(9):
    r=i//3
    c=i%3
    ax=axs[r][c].axis("off")
    actual=reverse_mapping[testY[i]]   
    predict=reverse_mapping[PRED[i]]    
    ax=axs[r][c].set_title(actual+'=='+predict)
    ax=axs[r][c].imshow(test_images[i])
plt.show()